# Layoffs Dataset Cleanup

In [29]:
# Import required libraries

import numpy as np 
import pandas as pd 
import requests
import json
import time

# Import API key
from resources.api_keys import geoapify_key

In [30]:
# Set path to input dataset
file_path = "Input_Data/layoffs.csv"

# read data from csv
layoffs_unsanitized_dataset = pd.read_csv(file_path)
layoffs_unsanitized_dataset

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised
0,Airtable,SF Bay Area,Product,237.0,0.27,2023-09-14,Series F,United States,1400.0
1,R3,New York City,Crypto,NaN,0.20,2023-09-14,Series A,United States,112.0
2,Akili Labs,Boston,Healthcare,NaN,0.40,2023-09-13,Post-IPO,United States,301.0
3,Google,SF Bay Area,Consumer,NaN,NaN,2023-09-13,Post-IPO,United States,26.0
4,Evolve,Denver,Travel,175.0,0.20,2023-09-12,Series F,United States,224.0
...,...,...,...,...,...,...,...,...,...
3017,Service,Los Angeles,Travel,NaN,1.00,2020-03-16,Seed,United States,5.1
3018,HopSkipDrive,Los Angeles,Transportation,8.0,0.10,2020-03-13,Unknown,United States,45.0
3019,Panda Squad,SF Bay Area,Consumer,6.0,0.75,2020-03-13,Seed,United States,1.0
3020,Tamara Mellon,Los Angeles,Retail,20.0,0.40,2020-03-12,Series C,United States,90.0


In [31]:
# Check NaN across the dataset
layoffs_unsanitized_dataset.isna().sum()

company                   0
location                  0
industry                  0
total_laid_off          991
percentage_laid_off    1050
date                      0
stage                     7
country                   0
funds_raised            307
dtype: int64

In [32]:
# Cleanup the dataset for blank values

#Check if the dataset has 'funds_raised','percentage_laid_off' columns, If yes drop those

if set(['funds_raised','percentage_laid_off']).issubset(layoffs_unsanitized_dataset.columns):

    # Drop funds raised and percentage laid off where the value is blank
    layoffs_unsanitized_dataset = layoffs_unsanitized_dataset.drop(['funds_raised','percentage_laid_off'], axis=1)

# There must be atleast 1 reported layoff in any company in this dataset
layoffs_unsanitized_dataset.total_laid_off = layoffs_unsanitized_dataset.total_laid_off.fillna(1)

# Layoffs should be whole numbers not floats
layoffs_unsanitized_dataset.total_laid_off = layoffs_unsanitized_dataset.total_laid_off.astype(int)

# Company with blank funding stage is set to 'Unknown'
layoffs_unsanitized_dataset.stage = layoffs_unsanitized_dataset.stage.fillna('Unknown')

# Print clean result data
layoffs_unsanitized_dataset.head()

,company,location,industry,total_laid_off,date,stage,country
0,Airtable,SF Bay Area,Product,237,2023-09-14,Series F,United States
1,R3,New York City,Crypto,1,2023-09-14,Series A,United States
2,Akili Labs,Boston,Healthcare,1,2023-09-13,Post-IPO,United States
3,Google,SF Bay Area,Consumer,1,2023-09-13,Post-IPO,United States
4,Evolve,Denver,Travel,175,2023-09-12,Series F,United States


In [33]:
# Add two new columns latitude and Longitude to be used later for Geoviews
layoffs_unsanitized_dataset["Lat"] = ""
layoffs_unsanitized_dataset["Lon"] = ""

# Define the API parameters
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

# Build a dictionary that will serve as Cache to not hit API for the same city, country combination
city_coordinates = {}


In [34]:
# Call GeoApify to get Latitute and Longitute for each city.
# Print a message to follow up the airport search
print("Starting City search")

# Loop through the cities_pd DataFrame and search coordinates for each city
for index, row in layoffs_unsanitized_dataset.iterrows():

    # Get the city's name & add ", Australia" to the string so geoapify finds the correct city
    city = row["location"] + ",",row["country"]

    # Add the current city to the parameters
    params["text"] = f"{city}"

    # Make the API request if city & country combination is not in Cache
    if city not in city_coordinates:
        # API is called as city not found in Cache
        print(f"Coordinates for {city} fetched...")
        try:
            response = requests.get(base_url, params=params)

            # Convert response to JSON
            response = response.json()
            city_coordinates[city] = [response["results"][0]["lat"], response["results"][0]["lon"]]
        except:
            print("City coorodinates not found. Skipping...")
            city_coordinates[city] = ['','']
            pass
    

    # Extract latitude and longitude
    layoffs_unsanitized_dataset.loc[index, "Lat"] = city_coordinates[city][0]
    layoffs_unsanitized_dataset.loc[index, "Lon"] = city_coordinates[city][1]
 

# Display sample data to confirm that the coordinates appear
layoffs_unsanitized_dataset.head()

Starting City search
Coordinates for ('SF Bay Area,', 'United States') fetched...
Coordinates for ('New York City,', 'United States') fetched...
Coordinates for ('Boston,', 'United States') fetched...
Coordinates for ('Denver,', 'United States') fetched...
Coordinates for ('Montreal,', 'Canada') fetched...
Coordinates for ('Berlin,', 'Germany') fetched...
Coordinates for ('Tallinn,', 'Estonia') fetched...
Coordinates for ('Amsterdam,', 'Netherlands') fetched...
Coordinates for ('Charlotte,', 'United States') fetched...
Coordinates for ('Jakarta,', 'Indonesia') fetched...
Coordinates for ('Tel Aviv,', 'United States') fetched...
Coordinates for ('Seattle,', 'United States') fetched...
Coordinates for ('Belo Horizonte,', 'Brazil') fetched...
Coordinates for ('Portland,', 'United States') fetched...
Coordinates for ('Accra,', 'Ghana') fetched...
Coordinates for ('Bengaluru,', 'India') fetched...
Coordinates for ('Chicago,', 'United States') fetched...
Coordinates for ('Brno,', 'Czech Repu

Coordinates for ('Bogota,', 'Colombia') fetched...
Coordinates for ('Durham,', 'United States') fetched...
Coordinates for ('Stockholm,', 'United States') fetched...
Coordinates for ('Buenos Aires,', 'Argentina') fetched...
Coordinates for ('Indore,', 'India') fetched...
Coordinates for ('Patna,', 'India') fetched...
Coordinates for ('Brisbane,', 'United States') fetched...
Coordinates for ('Bismarck,', 'United States') fetched...
Coordinates for ('Tokyo,', 'United States') fetched...
Coordinates for ('Singapore,', 'United States') fetched...
Coordinates for ('Grand Rapids,', 'United States') fetched...
Coordinates for ('Shanghai,', 'China') fetched...
Coordinates for ('Beau Vallon,', 'Seychelles') fetched...
Coordinates for ('Beijing,', 'United States') fetched...
Coordinates for ('Birmingham,', 'United States') fetched...
Coordinates for ('Burlington,', 'United States') fetched...
Coordinates for ('Bend,', 'United States') fetched...
Coordinates for ('Guadalajara,', 'Mexico') fetched

,company,location,industry,total_laid_off,date,stage,country,Lat,Lon
0,Airtable,SF Bay Area,Product,237,2023-09-14,Series F,United States,37.788497,-122.355847
1,R3,New York City,Crypto,1,2023-09-14,Series A,United States,40.737233,-73.862477
2,Akili Labs,Boston,Healthcare,1,2023-09-13,Post-IPO,United States,42.350804,-71.053673
3,Google,SF Bay Area,Consumer,1,2023-09-13,Post-IPO,United States,37.788497,-122.355847
4,Evolve,Denver,Travel,175,2023-09-12,Series F,United States,39.748625,-104.988623


In [39]:
layoffs_unsanitized_dataset

,company,location,industry,total_laid_off,date,stage,country,Lat,Lon
0,Airtable,SF Bay Area,Product,237,2023-09-14,Series F,United States,37.788497,-122.355847
1,R3,New York City,Crypto,1,2023-09-14,Series A,United States,40.737233,-73.862477
2,Akili Labs,Boston,Healthcare,1,2023-09-13,Post-IPO,United States,42.350804,-71.053673
3,Google,SF Bay Area,Consumer,1,2023-09-13,Post-IPO,United States,37.788497,-122.355847
4,Evolve,Denver,Travel,175,2023-09-12,Series F,United States,39.748625,-104.988623
...,...,...,...,...,...,...,...,...,...
3017,Service,Los Angeles,Travel,1,2020-03-16,Seed,United States,25.60289,-108.48095
3018,HopSkipDrive,Los Angeles,Transportation,8,2020-03-13,Unknown,United States,25.60289,-108.48095
3019,Panda Squad,SF Bay Area,Consumer,6,2020-03-13,Seed,United States,37.788497,-122.355847
3020,Tamara Mellon,Los Angeles,Retail,20,2020-03-12,Series C,United States,25.60289,-108.48095


In [37]:
# Store the sanitized data in a csv to use for data analysis and graph plot
layoffs_unsanitized_dataset.to_csv("Output_Data/Layoffs_Sanitized.csv", index=False)

####  I will use this clean dataset to find answers to the questions from this dataset. Please refer to "Layoffs_Dataset_Analysis.ipynb" notebook for the next part of the project